In [96]:
import pandas as pd
import numpy as np
import random
import operator
import math
from sklearn.datasets.samples_generator import make_blobs
from sklearn import datasets
import matplotlib.pyplot as plt
from matplotlib import style

x, y = make_blobs(n_samples=100, centers=5, n_features=2, random_state=1)
a, b = datasets.make_circles(n_samples=200, factor=.5, noise=.05)
c, d = datasets.make_moons(n_samples=200, noise=.05)

#df = pd.DataFrame.from_dict({'x':x})
df = pd.DataFrame(x)
# Number of Attributes
num_attr = len(df.columns) - 1

# Number of Clusters
k = 3

# Maximum number of iterations
MAX_ITER = 100

# Number of data points
n = len(df)

# Fuzzy parameter
m = 2.00


def initializeMembershipMatrix():
    membership_mat = list()
    for i in range(n):
        random_num_list = [random.random() for i in range(k)]
        summation = sum(random_num_list)
        temp_list = [x/summation for x in random_num_list]
        membership_mat.append(temp_list)
    return membership_mat


def calculateClusterCenter(membership_mat):
    cluster_centers = list()
    for j in range(k):
        x = list(membership_mat[j])
        #xraised = [e ** m for e in x]
        xraised = list(map(lambda x: x ** m, x))
        denominator = sum(xraised)
        temp_num = list()
        for i in range(len(xraised)):
            data_point = list(df.iloc[i])
            prod = [xraised[i] * val for val in data_point]
            temp_num.append(prod)
        numerator = map(sum, zip(*temp_num))
        center = [z/denominator for z in numerator]
        cluster_centers.append(center)
    return cluster_centers


def updateMembershipValue(membership_mat, cluster_centers):
    p = float(2/(m-1))
    for i in range(n):
        x = list(df.iloc[i])
        distances = [np.linalg.norm(list(map(operator.sub, x, cluster_centers[j]))) for j in range(k)]
        for j in range(k):
            den = sum([math.pow(float(distances[j]/distances[c]), p) for c in range(k)])
            if den == 0:
                print('{} index is 0.'.format(j))
            membership_mat[i][j] = float(1/den)       
    return membership_mat

def fuzzyCMeansClustering():
    # Membership Matrix
    membership_mat = initializeMembershipMatrix()
    curr = 0
    cluster_centers = None
    while curr <= MAX_ITER:
        previous = cluster_centers
        cluster_centers = calculateClusterCenter(membership_mat)
        membership_mat = updateMembershipValue(membership_mat, cluster_centers)
        for value in membership_mat:
            print(value)
        #cluster_labels = getClusters(membership_mat)
        isOptimal = True
        
        if previous == None:
            break
            
        '''    for center in cluster_centers:
                original = previous[center]
                current = self.centers[center]
                if np.sum( (current - original) / original * 100.0) > self.tolerance:
                    isOptimal = False
            if isOptimal:
                break
        '''
        
        curr += 1
    #print(membership_mat)
    #return cluster_labels, cluster_centers
    return cluster_centers


#labels, centers = fuzzyCMeansClustering()
centers = fuzzyCMeansClustering()
classification = centers.index(max(centers))
print(classification)

'''
memberships = initializeMembershipMatrix()
cent = calculateClusterCenter(memberships)
mem = updateMembershipValue(memberships, cent)
for value in mem:
    print(value)
'''

[0.4657432632923794, 0.45175460082464175, 0.08250213588297874]
[0.279436228813769, 0.6882269757645705, 0.032336795421660465]
[0.12073964437413756, 0.11127534601558889, 0.7679850096102735]
[0.2851115146117451, 0.40755655569077276, 0.3073319296974822]
[0.22572595686001432, 0.264363188193888, 0.5099108549460977]
[0.374192146540537, 0.499583983236044, 0.12622387022341885]
[0.38357500901662767, 0.49647374084963747, 0.11995125013373485]
[0.27669596422532305, 0.3556934912338815, 0.3676105445407955]
[0.1726141867316996, 0.18202819182575353, 0.6453576214425468]
[0.0552021130881345, 0.04824942180567355, 0.8965484651061919]
[0.23795906011448384, 0.2943666583538706, 0.4676742815316456]
[0.42522783976980255, 0.5154189921210355, 0.059353168109162]
[0.31750845722839155, 0.6092178167956414, 0.07327372597596714]
[0.27549364324609055, 0.6057449414005854, 0.11876141535332402]
[0.2628340489444308, 0.33209669215227516, 0.40506925890329415]
[0.3754119887188432, 0.5448552515680044, 0.07973275971315233]
[0.06

'\nmemberships = initializeMembershipMatrix()\ncent = calculateClusterCenter(memberships)\nmem = updateMembershipValue(memberships, cent)\nfor value in mem:\n    print(value)\n'